# Part IV: (Why) Is Python slow?
While Python is a very convenient and user-friendly language, it is considered as one of the slower ones.
Is this really the case? If yes, why? And how we can mitigate that? We will have a look!

In general applies, Python has an overhead of several function calls an invocations per executed command.
Naturally, this makes often reoccuring things less performant, as it is not compiler-optimizable.

The best example are **loops**.

Since Python is based on C, this is the most natural comparison:

## Loops
Loops are a good example for syntactics that should be avoided in performant code,

### Python

In [30]:
%%time
N = 10000000
data = list(range(N))

total = 0
for x in data:
    total += x
print("Sum:", total)

Sum: 49999995000000
CPU times: user 857 ms, sys: 160 ms, total: 1.02 s
Wall time: 1.01 s


### C

In [31]:
# !cat speed_comparison/sum_loop.c  # Comment in to show C code

In [32]:
!gcc -O3 speed_comparison/sum_loop.c -o sum_loop

In [33]:
%time
!./sum_loop

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 12.4 µs
Sum: 49999995000000

**Question**: Why is C so much faster?

In [34]:
from solutions import solution_speed2
print(solution_speed2)

C implements low-level arithmetics and high-performant, pre-compiled, and optimized loops. On top, Python has the comparably large interpreter overhead.


## Interpreter Overhead
Python introduces an interpreter overhead, compared to compiled languages, like C.
A simple addition, for example, is not only an arithmetic add, but involves several additional steps:

In [35]:
# The high-level byte-code is rather similar to C
import dis

def f(a, b):
    return a + b

dis.dis(f)

  4           0 RESUME                   0

  5           2 LOAD_FAST                0 (a)
              4 LOAD_FAST                1 (b)
              6 BINARY_OP                0 (+)
             10 RETURN_VALUE


While for C, this directly translates in assembler operations, the individual executions are interpreted in Python, which is essentially happening in a large while loop with different cases:
```
for (;;) {
    switch (opcode) {
        case LOAD_FAST:
            ...
        case BINARY_ADD:
            v = PyNumber_Add(v, w);
            ...
        ...
    }
}
```

A simple addition is implemented in C in three reister instructions: 
```
mov eax, edi
add eax, esi
ret
```
In Python however, several langauge-intrinsic things cause a large overhead (PER OPERATION!):
```
1) Parse a + b -> AST; convert to bytecode (LOAD_FAST a, LOAD_FAST b, BINARY_OP +).
2) Interpreter runs BINARY_OP; may specialize to ADD_INT/ADD_FLOAT based on observed types.
3) Dispatch: call PyNumber_Add(a, b), an abstracted c_call from the CPython implementation..
4) Slots: try a.__add__(b); if NotImplemented, try b.__radd__(a); else TypeError.
5) Types match! -> int + int: arbitrary-precision in PyLongObject; fast word add then fallback to multi-precision on overflow; allocate new PyLong; normalize; update refcounts.
6) In case of float: float + float: add C doubles (IEEE-754); allocate new PyFloatObject; handle inf/NaN per platform.
7) Mixed types: numeric coercion rules (e.g., int + float -> float).
8) In-place +=: uses __iadd__ if defined, else normal add; immutables (int, float) always allocate new. Under the hood: call CPzthon PyNumber_InPlaceAdd;
9) Memory/GIL: objects allocated via CPython allocators; refcounted; GIL ensures interpreter safety -> prevents that during all those steps, another thread/operation alters the currently used objects.
```

Especially 9) is one of the main arguments for the GIL!

**Take-away**: Instead of three low-level arithmetics instructions, a simple addition in Python calls a whole stack of function calls, allocations, validations, etc. All this happens automatically in the background and makes the language very convenient to use, BUT at the cost of reduced performance!

## Let's make it even worse: Print()

### Python

In [1]:
%%time
!python3 speed_comparison/sum_loop_print.py > output_py.txt

CPU times: user 121 ms, sys: 43.5 ms, total: 165 ms
Wall time: 8.7 s


### C

In [3]:
!gcc -O3 speed_comparison/sum_loop_print.c -o sum_loop_print

speed_comparison/sum_loop_print.c: In function ‘main’:
speed_comparison/sum_loop_print.c:9:18: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘long long int’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
    9 |         printf("%d\n", total);
      |                 ~^     ~~~~~
      |                  |     |
      |                  int   long long int
      |                 %lld


In [4]:
%%time
!./sum_loop_print > output_C.txt

CPU times: user 28.4 ms, sys: 9.39 ms, total: 37.8 ms
Wall time: 1.53 s


Note that C also suffers a lot from the prints! In general applies: **Only print what you really need and remove/hide e.g. debug outputs etc in production software.**

## How about if?
In C(++), avoiding branching within loops to reduce bad prediction occurences is a common optimization strategy.
This is the only "good news" in Python: we do not need to care (mostly).

Due to the GIL and the generally slower execution, branch predictions are not really making a difference, if we do not use lowest level things.

## Optimization Strategies and Good Practices
As we have seen, default Python is very slow, especially with loops.
But often, this can be covered and mitigated with good coding practices.

In the following, we will have a look at different opimization strategies.

### Starting Point: A simple nested for loop with an inplace addition of a division

In [39]:
%%time
total = 0.0
for i in range(9999):
    for j in range(1, 9999):
        total += (float(i) / j)
print(total)

489223499.9991652
CPU times: user 14.7 s, sys: 0 ns, total: 14.7 s
Wall time: 14.7 s


### Always the first good option to try: Using NumPy
The first thing you should get used to is using NumPy!
It is in many cases directly the best we can get, at least without high additional effort.

In [40]:
%%time
import numpy as np
i = np.arange(9999)
j = np.arange(1,9999)
print(np.divide.outer(i,j).sum())

489223499.99918354
CPU times: user 203 ms, sys: 0 ns, total: 203 ms
Wall time: 201 ms


**This is enormeous improvement! As we will see, this will be hard to beat!**

But we will try.

### Let's Optimize!
(Examples based on: https://stackoverflow.com/questions/8097408/why-python-is-so-slow-for-a-simple-for-loop)

#### Let's wrap everything in a function 

In [43]:
# Step 1: Let's make it a function!
def f1(num):
    total = 0.0
    for i in range(num): 
        for j in range(1, num):
            total += (float(i) / j)
    return total

In [44]:
%%time
print(f1(9999))

489223499.9991652
CPU times: user 6.63 s, sys: 1.77 ms, total: 6.63 s
Wall time: 6.64 s


**Question**: Any idea, why this is faster than the version before (without a function)?

Your answer:

In [45]:
from solutions import solution_speed4
print(solution_speed4)

When you define a function, Python compiles its bytecode once and it is reused each time you call it.
For the global code, the entire loop is recompiled, which can cause an overhead.
On top, inside the function, the variables are local, which is faster to access than global variables.
Because in the global scope, variables are internally stored in a dictionary, the globals(), and accessing them is essentially a hash lookup. This is in general fast, but in tight loops, this is done thousands of times and adds up. This then can make loops in the global scope slightly slower.

In a function, the variables are as well saved in a dict, the locals(), but accessing works differently. For each function call, a frame object pointer is created, which represents one execution fram and holds the functions local variables, arguments, and execution state. Inside this frame object, the variables are stored in optimized C arrays (so-called fast locals array) and can be accessed through a simple array 

In [46]:
# Showcase: Dict lookup instead of fast array access -> THIS IS SLOWER
def f1_b(num):
    local_vars = locals()
    local_vars['total'] = 0.0
    for i in range(num):
        for j in range(1, num):
            local_vars['total'] += (float(i) / j)  # Dict lookup instead of LOAD_FAST/STORE_FAST
    return local_vars['total']

In [51]:
%%time
print(f1_b(9999))

489223499.9991652
CPU times: user 10.1 s, sys: 3.84 ms, total: 10.1 s
Wall time: 10.1 s


#### Removing the Manual Cast

In [52]:
def f2(num):
    total = 0.0
    for i in range(num): 
        for j in range(1, num):
            total += (i / j)
    return total

In [53]:
%%time
print(f2(9999))

489223499.9991652
CPU times: user 3.74 s, sys: 2.44 ms, total: 3.75 s
Wall time: 3.75 s


In [54]:
from solutions import solution_speed5
print(solution_speed5)

Removing the cast allows Python to do internal optimization, which is often based on C implementations and therefore faster. This shows that if you are not entirely aware of how things are handled, it can be better to let it just do its job. But: we can do better!


#### Using NumPy
Typically the default way!

In [55]:
import numpy as np
def f3(num):
    i = np.arange(num, dtype=np.float64)
    j = np.arange(1, num, dtype=np.float64)
    return np.divide.outer(i, j).sum()

In [56]:
%%time
print(f3(9999))

489223499.99918354
CPU times: user 74.8 ms, sys: 59.3 ms, total: 134 ms
Wall time: 133 ms


Wow, already a very nice speedup without any big effort! And have a look, it is again faster than the initial NumPy version without a function.

#### Precision
If not necessary, reducting the precision can help to optimize performance. But this highly depends on the problem!

In [57]:
# Reducing precision, if not needed
# See: https://numpy.org/doc/stable/user/basics.types.html
import numpy as np
def f4(num):
    i = np.arange(num, dtype=np.float32)
    j = np.arange(1, num, dtype=np.float32)
    return np.divide.outer(i, j).sum()

In [58]:
%%time
print(f4(9999))

489223300.0
CPU times: user 41 ms, sys: 35.2 ms, total: 76.2 ms
Wall time: 74.1 ms


#### Using specialized (NumPy) Functions
For some purposes, more optimized functions are available, such as np.true_divide in this case

In [59]:
import numpy as np
def f5(num):
    x=np.ones(num-1)
    y=np.arange(1,num)
    return np.sum(np.true_divide(x,y))*np.sum(y)

In [60]:
%%time
print(f5(9999))

489223499.99918455
CPU times: user 2 ms, sys: 30 µs, total: 2.03 ms
Wall time: 1.35 ms


That is again a factor ~50! 

#### Be Smart!
This is the most shittiest tip, but sometimes, it really helps, if you are just smart!
For some problems, special algorithms or mathematical formulas exist.
In this case, there is an explicit mathematical formulation of the problem!

In [61]:
# Being smart is typically the hardest optimization strategy (at least for me...)!
import numpy as np
def f6(num):
    return np.sum(np.reciprocal(np.arange(1, num).astype(np.float64))) * num*(num-1)/2

In [62]:
%%time
print(f6(9999))

489223499.9991845
CPU times: user 1.19 ms, sys: 3 µs, total: 1.2 ms
Wall time: 894 µs


**NOTE: Since it is very fast, the restult can fluctuate. Run it multiple times!**

**This is a very nice speedup!**

### Wrap-Up

The default option for optimizing numerical operations is NumPy and vectorization!
Python loops execute one operation at a time in user space, with no SIMD (vectorized) optimization.

NumPy instead delegate the entire loop to compiled C code, which runs at native speed and can use vector instructions (AVX, SSE).
If you are interested, you can have a look at the optimizations for fast loops: 

https://github.com/numpy/numpy/blob/main/numpy/_core/src/umath/fast_loop_macros.h

This let's us profit from fast C implementations while coding on a higher, more simplified abstraction level.


Furthermore, we have seen: Other optimizations can be possible and useful, but the more specific it gets, the more problem dependend the approach is and no general recommendations can be given anymore.

**Question**: What else do you recognize?

Your answer:

In [63]:
from solutions import solution_speed3
print(solution_speed3)

The results differ in some late digits! This is due to different precisions. This, you have always to take into account for your computations. E.g. if you need high precision, you MUST use float64, there is no way round and no way to optimize this.


### Other Optimizations
- Often it is possible to reduce the number of operations. Mainly with columnar data ("vector additions"). Numpy can handle that easily!
- Using dicts, see below
- Use native C code
- Of course, there are many many more...

#### Using Dicts
Of course, this is a very generic example, but ir proves the point that dicts can be very nice!

In [64]:
import time
import random
import string

# Generate random words
N = 100000
words = [''.join(random.choices(string.ascii_lowercase, k=5)) for _ in range(N)]

# Select random sample to query
queries = random.choices(words, k=50_000)

# 1. Using dict
d = {w: random.randint(0, 100) for w in words}
start = time.perf_counter()
_ = [d[q] for q in queries]  # O(1) lookup
end = time.perf_counter()
print(f"dict lookup: {end - start:.3f} s")

# 2. Using list of tuples
pairs = list(d.items())
start = time.perf_counter()
_ = [v for q in queries for k, v in pairs if k == q]  # O(n) per lookup
end = time.perf_counter()
print(f"list lookup: {end - start:.3f} s")

dict lookup: 0.005 s
list lookup: 93.794 s


### Native C Code
(Example taken from Sebastien Ponce/CERN: https://gitlab.cern.ch/sponce/tcsccourse)

As already mentioned, Python can be used as a wrapper for large C(++) frameworks. This is a very common approach in the LHC environment and good practice.

On top shown above, you can use precompiled C code within Python.
This can come in very handy and is often used, when performance/runtime is a strict requirement.

The given example calculates and plots https://en.wikipedia.org/wiki/Mandelbrot_set:
```
The Mandelbrot set is a two-dimensional set that is defined in the complex plane as the complex numbers c {\displaystyle c} for which the function f c ( z ) = z 2 + c {\displaystyle f_{c}(z)=z^{2}+c} does not diverge to infinity when iterated starting at z = 0 {\displaystyle z=0}, i.e., for which the sequence f c ( 0 ) {\displaystyle f_{c}(0)}, f c ( f c ( 0 ) ) {\displaystyle f_{c}(f_{c}(0))}, etc., remains bounded in absolute value.
```

In [ ]:
!python3 speed_comparison/mandel/mandel_py.py --out mandel_py.png --bench --backend numpy

If you want, you can try out the other backends as well.

In [ ]:
!python3 speed_comparison/mandel/mandel_py.py -h

In [ ]:
# Get the C code
!git clone https://gitlab.cern.ch/sponce/tcsccourse.git

In [ ]:
!cd tcsccourse/exercises/exercise5 && make libmandel.so

In [ ]:
!cd tcsccourse/exercises/exercise5 && export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:. && python3 mandel.py

**Take-away:** Even with NumPy, Python is in general slower than low-level languages like C(++) or Rust, but if used correctly, we can get in range while maintaining most of Python's ease of use!

## Bonus: Implicit vs Explicit -- Part II 
Remember the first example?
When writing Python code, implicit concepts like chanining and list comprehensions can be helpful and reduce repetitions and bloated code.
However, readibility suffers a lot, why it is **not** recommended for beginners.

**Question**: What do you thing is faster: implicit or explicit?

Your answer:

In [ ]:
from solutions import solution_speed0
print(solution_speed0)

Try it out:

In [ ]:
import time
from functools import wraps

def timing_decorator(func):
    @wraps(func)  # This decorator preserves the original function's metadata
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {(end - start):.4f} seconds")
        return result
    return wrapper

def process(value):
    print(f"Processing {value}")
    return value.upper()

ready = True
data = "apple"
default = "banana"
N = 1000

implicit_times = []
explicit_times = []

@timing_decorator
def implicit():    
    # --- Implicit and complex ---
    start = time.perf_counter()
    result = process(data or default) if ready else None
    print("Implicit: ", result)

@timing_decorator
def explicit():
    if ready:
        if data:
            result = process(data)
        else:
            result = process(default)
    else:
        result = None
    print("Explicit: ", result)
implicit()
explicit()

**Question:** Why is the following implementation faster?

In [ ]:
import time
import statistics

def process(value):
    return value.upper()

ready = True
data = "apple"
default = "banana"
N = 1000

implicit_times = []
explicit_times = []

for _ in range(N):
    # --- Implicit and complex ---
    start = time.perf_counter()
    result = process(data or default) if ready else None
    implicit_times.append(time.perf_counter() - start)

    # --- Explicit and simple ---
    start = time.perf_counter()
    if ready:
        if data:
            result = process(data)
        else:
            result = process(default)
    else:
        result = None
    explicit_times.append(time.perf_counter() - start)

print(f"Implicit avg: {statistics.mean(implicit_times):.8f}s ± {statistics.stdev(implicit_times):.8f}")
print(f"Explicit avg: {statistics.mean(explicit_times):.8f}s ± {statistics.stdev(explicit_times):.8f}")

Your answer:

In [ ]:
from solutions import solution_speed1
print(solution_speed1)

------------------------